In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from homebrewedFunctions.functions import *
stack_dfs = pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], low_memory =False).sort_index()
panel_dfs_dict = {k:stack_dfs[stack_dfs.index.get_level_values(2)==k].reset_index().set_index(["State","Year"]).sort_index() for k in stack_dfs.index.get_level_values(2).unique()}
for key, df in panel_dfs_dict.items():
    for col in df.columns:
        if not is_numeric_dtype(df[col]):
            try:
                df[col] = pd.to_numeric(df[col], errors = "coerce")
                df[col] = df[col].fillna(0)
                df[col] = df[col].astype(int)
            except:
                pass
        else:
            df[col] = df[col].fillna("")
    df["1TOTAL INCOME"] = df["1INDIVIDUAL INCOME"].add(df["1CORPORATE INCOME"])
    df["1PROPERTY AND SPECIAL ASSESSMENTS"] = df[["1PROPERTY", "1SPECIAL ASSESSMENTS"]].sum(axis = 1)
    df["1DEFICIT"] = df["1EXPENDITURE"].sub(df["1GENERAL REVENUE"])

/tmp/ipykernel_6084/3822857466.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["1TOTAL INCOME"] = df["1INDIVIDUAL INCOME"].add(df["1CORPORATE INCOME"])
/tmp/ipykernel_6084/3822857466.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["1PROPERTY AND SPECIAL ASSESSMENTS"] = df[["1PROPERTY", "1SPECIAL ASSESSMENTS"]].sum(axis = 1)
/tmp/ipykernel_6084/3822857466.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

In [2]:
efwgdp = pd.read_csv("EFWGDPTaxes.csv", parse_dates = ["Year"]).set_index(["State", "Year"]).sort_index()
efwgdp[["EFW", "GDP"]]

EFW           GDP
State Year                          
AK    1985-01-01  8.04           NaN
      1986-01-01   NaN           NaN
      1987-01-01   NaN           NaN
      1988-01-01   NaN           NaN
      1989-01-01   NaN           NaN
...                ...           ...
WY    2019-01-01  8.12  3.997140e+10
      2020-01-01  7.97  3.667550e+10
      2021-01-01   NaN  4.217620e+10
      2022-01-01   NaN  4.908060e+10
      2023-01-01   NaN  5.017160e+10

[1950 rows x 2 columns]

In [3]:
map_keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
        "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" ,
        "TAXES", "OTHER TAXES", "CURRENT CHARGES", "OTHER CHARGES", 'INDIVIDUAL INCOME', 'CORPORATE INCOME', #'TOTAL INCOME',
        "PROPERTY", "SPECIAL ASSESSMENTS"]#"PROPERTY AND SPECIAL ASSESSMENTS",  
        # "SALES AND GROSS RECEIPTS", "GENERAL SALES", "MOTOR FUEL", "GAS SUPPLY", "EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]	

keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
        "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" ,
        "TAXES", "OTHER TAXES", 'PROPERTY', 'SPECIAL ASSESSMENTS', 'PROPERTY AND SPECIAL ASSESSMENTS', 'SALES AND GROSS RECEIPTS', 'GENERAL SALES', 'SELECTIVE SALES', 'MOTOR FUEL', 'ALCOHOLIC BEVERAGE', 
        'TOBACCO PRODUCTS', 'PUBLIC UTILITIES', 'OTHER SELECTIVE SALES', 'INDIVIDUAL INCOME', 'CORPORATE INCOME', "TOTAL INCOME", 'MOTOR VEHICLE LICENSE', 
        'OTHER TAXES', 'CHARGES AND MISCELLANEOUS GENERAL  REVENUE', 'CURRENT CHARGES', 'EDUCATION', 'INSTITUTIONS  OF HIGHER EDUCATION', 
        'SCHOOL LUNCH SALES (GROSS)', 'HOSPITALS', 'HIGHWAYS', 'AIR TRANSPORTATION (AIRPORTS)', 'PARKING FACILITIES', 'SEA AND INLAND PORT FACILITIES', 
        'NATURAL RESOURCES', 'PARKS AND RECREATION', 'HOUSING AND COMMUNITY DEVELOPMENT', 'SEWERAGE', 'SOLID WASTE MANAGEMENT', 'OTHER CHARGES', 
        'MISCELLANEOUS GENERAL REVENUE', 'INTEREST EARNINGS',  'SALE OF PROPERTY', 'OTHER GENERAL REVENUE', 'UTILITY REVENUE', 
        'WATER SUPPLY', 'ELECTRIC POWER', 'GAS SUPPLY', 'TRANSIT', 'LIQUOR STORE REVENUE', 'INSURANCE TRUST REVENUE', 'UNEMPLOYMENT COMPENSATION', 
        'EMPLOYEE RETIREMENT', "WORKERS' COMPENSATION", 'OTHER INSURANCE TRUST REVENUE',"EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]
keys = ["1" + k for k in keys]

In [4]:
plot_dfs = {}
keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
        "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" ,
        "TAXES", "OTHER TAXES", "CURRENT CHARGES", "OTHER CHARGES", 'INDIVIDUAL INCOME', 'CORPORATE INCOME', 'TOTAL INCOME',
        "PROPERTY", "SPECIAL ASSESSMENTS", "PROPERTY AND SPECIAL ASSESSMENTS",  
        "SALES AND GROSS RECEIPTS", "GENERAL SALES", "MOTOR FUEL", "GAS SUPPLY", "EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]	
def set_plot_dfs(panel_dfs_dict, keys, efwgdp):
    keys = ["1" + k for k in keys]
    for key in panel_dfs_dict.keys():
        plot_dfs[key] = {}
        plot_dfs[key]["Level"] = panel_dfs_dict[key][keys].mul(10**3).copy()
        plot_dfs[key]["Level"].rename(columns = {k:k.replace("1", "").title() for k in plot_dfs[key]["Level"].keys()}, inplace = True)
    #     plot_dfs[key]["Level"]["Total Income"] = plot_dfs[key]["Level"]["Individual Income"].add(plot_dfs[key]["Level"]["Corporate Income"])
    #     plot_dfs[key]["Level"]["Property and Special Assessments"] = plot_dfs[key]["Level"][["Property", "Special Assessments"]].sum(axis = 1)
    #     plot_dfs[key]["Level"]["Deficit"] = plot_dfs[key]["Level"]["Expenditure"].sub(plot_dfs[key]["Level"]["General Revenue"])
        plot_dfs[key]["Level"]["GDP"] = efwgdp["GDP"]
        plot_dfs[key]["Level"]["Population"] = efwgdp["Population"]
        plot_dfs[key]["Percent of General Revenue"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["General Revenue"]).mul(100))    
        plot_dfs[key]["Percent of GDP"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["GDP"]).mul(100))
        plot_dfs[key]["Value of Per Capita"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["Population"]))    
        for key in plot_dfs.keys():
            plot_dfs[key]["Level"]["EFW"] = efwgdp["EFW"]
    return plot_dfs
plot_dfs = set_plot_dfs(panel_dfs_dict, keys, efwgdp)


In [5]:

for key, p_dfs in plot_dfs.items():
    for p_dfskey, df in p_dfs.items():
        map_figs = {}
        create_scatter_dropdown(df, 
                                filename = f"outputs/ScatterAppIncomePropertyAssessmentSalesFuelTaxesPctTotalRevenue{key}{p_dfskey}.html", 
                                show_fig = False)
        for name in df.keys():
            map_figs[name] = create_map(df.reset_index(), name, time_name = "Year")
        combined_fig = combine_map_figs(map_figs)
        combined_fig.write_html(f"outputs/AllMapsByVariableAndYear{key}{p_dfskey}.html")



In [6]:
plot_dfs = {}
keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
        "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" , "INTERGOVERNMENTAL REVENUE",
        "TAXES", "OTHER TAXES", 'PROPERTY', 'SPECIAL ASSESSMENTS', 'PROPERTY AND SPECIAL ASSESSMENTS', 'SALES AND GROSS RECEIPTS', 'GENERAL SALES', 'SELECTIVE SALES', 'MOTOR FUEL', 'ALCOHOLIC BEVERAGE', 
        'TOBACCO PRODUCTS', 'PUBLIC UTILITIES', 'OTHER SELECTIVE SALES', 'INDIVIDUAL INCOME', 'CORPORATE INCOME', "TOTAL INCOME", 'MOTOR VEHICLE LICENSE', 
        'OTHER TAXES', 'CHARGES AND MISCELLANEOUS GENERAL  REVENUE', 'CURRENT CHARGES', 'EDUCATION', 'INSTITUTIONS  OF HIGHER EDUCATION', 
        'SCHOOL LUNCH SALES (GROSS)', 'HOSPITALS', 'HIGHWAYS', 'AIR TRANSPORTATION (AIRPORTS)', 'PARKING FACILITIES', 'SEA AND INLAND PORT FACILITIES', 
        'NATURAL RESOURCES', 'PARKS AND RECREATION', 'HOUSING AND COMMUNITY DEVELOPMENT', 'SEWERAGE', 'SOLID WASTE MANAGEMENT', 'OTHER CHARGES', 
        'MISCELLANEOUS GENERAL REVENUE', 'INTEREST EARNINGS',  'SALE OF PROPERTY', 'OTHER GENERAL REVENUE', 'UTILITY REVENUE', 
        'WATER SUPPLY', 'ELECTRIC POWER', 'GAS SUPPLY', 'TRANSIT', 'LIQUOR STORE REVENUE', 'INSURANCE TRUST REVENUE', 'UNEMPLOYMENT COMPENSATION', 
        'EMPLOYEE RETIREMENT', "WORKERS' COMPENSATION", 'OTHER INSURANCE TRUST REVENUE',"EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]
plot_dfs = set_plot_dfs(panel_dfs_dict, keys, efwgdp)


In [7]:
for key, p_dfs in plot_dfs.items():
    figs = {key: line_dropdown(dataframe) for key, dataframe in p_dfs.items()}
    fig = dict_of_figs_to_dropdown_fig(figs, show_fig = False,
                                    use_sliders = True)
    # fig.show()
    fig.write_html(f"outputs/LineplotAppStateFinancesAsPercentRevenuePercentGDPAndPerCapitaFigs{key}.html")

# fig = line_dropdown(pct_df)
# fig.write_html("line_dropdown.html")

In [11]:
start_year = 2005
gov_keys = ["Local government amount",'State & local government amount', 'State government amount']
states = plot_dfs[gov_keys[0]]["Level"].index.get_level_values("State").unique()
areas = {"Revenue Source by Government" : ["General Revenue From Own Sources", "From Federal Government", "From State Government", "From Local Governments"],
         
    #  Omit revenue sources according to distinction of subcomponents in revenue data 
    #  Some sources additionally needed to be removed as their values perfectly explained
    #  accounting that sums to greater than 100 percent
     
         "Taxes": ["Intergovernmental Revenue", 'Property', 'Sales And Gross Receipts', 
                   'Individual Income', 'Corporate Income', 'Motor Vehicle License', 'Other Taxes', 
                #    'Charges And Miscellaneous General  Revenue', 
                   'Current Charges', 
                #    'Education', 'Institutions  Of Higher Education', 'School Lunch Sales (Gross)',
                #      'Hospitals', 'Highways', 'Air Transportation (Airports)', 'Parking Facilities', 'Sea And Inland Port Facilities', 'Natural Resources',
                #       'Parks And Recreation', 'Housing And Community Development', 'Sewerage', 'Solid Waste Management', 'Other Charges', 
                    #   'Miscellaneous General Revenue', 
                      'Interest Earnings', 'Special Assessments', 'Sale Of Property', 'Other General Revenue']}
                    #   'Utility Revenue']}
                    #   'Water Supply', 'Electric Power', 'Gas Supply', 'Transit', 
                    # 'Liquor Store Revenue',
                #    'Insurance Trust Revenue',
                #    'Unemployment Compensation', 'Employee Retirement', "Workers' Compensation", 'Other Insurance Trust Revenue']}
figs = {}
for key in gov_keys:
    figs[key] = {}
    for form in ["Level", "Percent of General Revenue", "Percent of GDP", "Value Per Capita"]:
        figs[key][form] = {}
        df = plot_dfs[key][form].copy()
        df = df.reset_index().melt(id_vars=["State","Year"], 
            var_name="Name", 
            value_name="Value")
        df = df.set_index(["State", "Year"]).round(2)
        for components_group, components in areas.items():


            figs[key][form][components_group] = {}

            for state in states:
                plot_df = df.loc[state].reset_index()#.loc[start_year:].reset_index()
                plot_df = plot_df[plot_df["Name"].isin(components)].dropna()
                plot_df["Value"] = pd.to_numeric(plot_df["Value"])
                title_key = f"{form}<br>{key} {components_group}<br>{state}"
                px_fig = px.area(
                    plot_df, x="Year", y="Value", color="Name", title = title_key)

                figs[key][form][components_group][state] = px_fig
            figs[key][form][components_group]  = dict_of_figs_to_dropdown_fig(figs[key][form][components_group] , show_fig = False,
                                        use_sliders = True)

            figs[key][form][components_group].write_html(f"outputs/{key}{form}{components_group}Figs.html")
